In [1]:
import polars_ds as pld
import polars as pl
import numpy as np

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.676152,0.82058,0,10000,-10000,1,0.806838,"""a"""
0.841471,1,"""a""",0.46256,0.643049,1,10001,-9999,1,0.938433,"""a"""
0.909297,2,"""a""",0.108087,0.706099,2,10002,-9998,0,0.846328,"""a"""
0.14112,3,"""a""",0.155237,0.55966,3,10003,-9997,0,0.737822,"""a"""
-0.756802,4,"""a""",0.386844,0.161174,4,10004,-9996,1,0.284666,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
df.select(
    pl.col("f").num.rfft()
).head()

f
list[f64]
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# Least Square (Linear Regression)
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)

y
list[f64]
"[2.0, -1.0]"


In [6]:
df.select(
    pl.col("y").num.lstsq_report(
        pl.col("x1"), pl.col("x2"),
        add_bias = False
    ).alias("report")
).unnest("report")

feat_idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [7]:
df.lazy().select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
).collect()

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [9]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    (pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False, return_pred=True)).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,0.0
1,10001,-9999,-9999.0,0.0
2,10002,-9998,-9998.0,0.0
3,10003,-9997,-9997.0,0.0
4,10004,-9996,-9996.0,0.0


In [10]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)


dummy,coeffs
str,list[f64]
"""a""","[2.0, -1.0]"
"""b""","[2.0, -1.0]"


In [11]:
# Rolling regression
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [12]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


In [13]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.bce(pl.col("predicted")).alias("log loss"),
    pl.col("actual").metric.binary_metrics_combo(pl.col("predicted")).alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.329687,0.979192,0.513655,0.504536,0.254527,0.512649,0.50632
"""a""",0.326209,0.973953,0.513448,0.511805,0.256313,0.516627,0.513395


# Str Extension

In [14]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [15]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""to"""
"""hello"""
"""world"""
"""church"""
"""going"""


In [16]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""world"""
"""church"""
"""hello"""
"""go"""


In [17]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [18]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [19]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [20]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pl.col("word").str2.levenshtein_filter("world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [21]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [22]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [23]:

df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,null,0.833333,0.966667
0.428571,0.333333,0.272727,null,0.0,0.444444
0.111111,0.111111,0.090909,null,0.444444,0.5
0.875,0.666667,0.545455,null,0.25,0.527778
0.75,0.777778,0.454545,null,0.25,0.527778


# Stats Extension

In [24]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.406361
-0.461633
1.690807


In [25]:
# Genenrate random sample, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
-0.406361,0.393981
-0.461633,0.056448
1.690807,0.1436


In [26]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.406361,"""w"""
-0.461633,"""R"""
1.690807,"""BB"""


In [27]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.406361,"""3hKco"""
-0.461633,"""DQ8N8"""
1.690807,"""8lMkY"""


In [28]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.sample_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.sample_uniform(low = 0., high = None, respect_null=True).alias("test2")
).head()

a,test1,test2
f64,f64,f64
null,-2.821919,null
null,-0.229692,null
-0.406361,-0.30437,1.990911
-0.461633,0.746839,1.497461
1.690807,0.817995,3.899714


In [29]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.sample_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-1.762153,0.078255,7.65309,0.021785


In [30]:
size = 5000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.876752,0.922415,0,2
1,0.007555,0.541163,0,5
2,0.406977,0.40947,4,0
0,0.551754,0.960711,1,7
1,0.321661,0.090088,4,3


In [31]:
# In dataframe statistical tests!
df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.520389,0.602804}","{32.325411,0.644077}","{0.5718,0.683112}"


In [32]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-1.734957,0.082841}","{26.106608,0.887511}","{0.489862,0.743213}"
1,"{1.521324,0.128273}","{39.890696,0.301196}","{0.700162,0.591837}"
2,"{-0.678035,0.497797}","{24.939887,0.917228}","{0.372588,0.828309}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [33]:
import polars_ds as pld
df = pl.DataFrame({
    "id": range(1000),
    "val1": np.random.random(size=1000), 
    "val2": np.random.random(size=1000), 
    "val3": np.random.random(size=1000),
    "r": np.random.random(size=1000),
    "rh": np.random.random(size=1000)*10,
})

In [34]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pld.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head()

id,val1,val2,val3,r,rh,nb_l_inf_cnt
i64,f64,f64,f64,f64,f64,u32
0,0.830397,0.234314,0.260266,0.013577,8.315271,9
1,0.437053,0.833232,0.504109,0.700449,1.823099,10
2,0.064311,0.911811,0.060288,0.985172,8.29957,8
3,0.744243,0.983393,0.691508,0.505907,4.122107,2
4,0.705555,0.197482,0.954982,0.682109,7.82336,5


In [35]:
df.with_columns(
    pld.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,val1,val2,val3,r,rh,nb_l1_r_cnt
i64,f64,f64,f64,f64,f64,u32
0,0.830397,0.234314,0.260266,0.013577,8.315271,270
1,0.437053,0.833232,0.504109,0.700449,1.823099,35
2,0.064311,0.911811,0.060288,0.985172,8.29957,19
3,0.744243,0.983393,0.691508,0.505907,4.122107,258
4,0.705555,0.197482,0.954982,0.682109,7.82336,115


In [36]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pl.col("id").num.knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends")
).head()

id,val1,val2,val3,r,rh,best friends
i64,f64,f64,f64,f64,f64,list[u64]
0,0.830397,0.234314,0.260266,0.013577,8.315271,"[0, 692, … 170]"
1,0.437053,0.833232,0.504109,0.700449,1.823099,"[1, 311, … 322]"
2,0.064311,0.911811,0.060288,0.985172,8.29957,"[2, 55, … 246]"
3,0.744243,0.983393,0.691508,0.505907,4.122107,"[3, 699, … 612]"
4,0.705555,0.197482,0.954982,0.682109,7.82336,"[4, 794, … 81]"


In [37]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
df.with_columns(
    pl.col("id").num.query_radius_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).list.slice(offset = 1).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

id,val1,val2,val3,r,rh,best friends,best friends count
i64,f64,f64,f64,f64,f64,list[u64],u32
0,0.830397,0.234314,0.260266,0.013577,8.315271,"[692, 707, … 126]",99
1,0.437053,0.833232,0.504109,0.700449,1.823099,"[311, 148, … 597]",92
2,0.064311,0.911811,0.060288,0.985172,8.29957,"[55, 254, … 198]",41
3,0.744243,0.983393,0.691508,0.505907,4.122107,"[699, 343, … 743]",61
4,0.705555,0.197482,0.954982,0.682109,7.82336,"[794, 934, … 282]",58


In [38]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pl.col("id").num.knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,val1,val2,val3,r,rh,idx,dist
i64,f64,f64,f64,f64,f64,list[u64],list[f64]
0,0.830397,0.234314,0.260266,0.013577,8.315271,"[0, 692, … 170]","[0.0, 0.003897, … 0.00671]"
1,0.437053,0.833232,0.504109,0.700449,1.823099,"[1, 311, … 322]","[0.0, 0.005956, … 0.009372]"
2,0.064311,0.911811,0.060288,0.985172,8.29957,"[2, 55, … 246]","[0.0, 0.004423, … 0.00995]"
3,0.744243,0.983393,0.691508,0.505907,4.122107,"[3, 699, … 612]","[0.0, 0.016528, … 0.022786]"
4,0.705555,0.197482,0.954982,0.682109,7.82336,"[4, 794, … 81]","[0.0, 0.005788, … 0.00612]"


In [39]:
# Filter to only points near the given point
df.filter(
    pld.query_radius(
        [0.5, 0.5, 0.5],
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
1,0.437053,0.833232,0.504109,0.700449,1.823099
5,0.455281,0.196307,0.619922,0.640082,9.703718
7,0.806839,0.683767,0.437859,0.378225,3.276941
8,0.248278,0.623837,0.371282,0.20446,3.528847
10,0.658954,0.366854,0.304322,0.334357,8.798462


In [40]:
# Haversine distance is available when dimension is 2
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        r = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
135,0.48709,0.573119,0.589296,0.531008,7.575682
137,0.483687,0.446175,0.052932,0.260512,7.253885
234,0.531116,0.536306,0.62751,0.747648,3.934703
258,0.493671,0.483423,0.82552,0.155978,5.17704
295,0.481641,0.55431,0.010235,0.02266,3.592055


In [41]:
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
137,0.483687,0.446175,0.052932,0.260512,7.253885
258,0.493671,0.483423,0.82552,0.155978,5.17704
328,0.526751,0.541864,0.315757,0.544857,9.57898
415,0.484797,0.494824,0.201883,0.110086,9.94729
564,0.523396,0.471598,0.851707,0.378799,6.20172


In [42]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pl.col("id").num.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        # This function 
        r = 0.05, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u64],u32
0,"[0, 601, … 796]",144
1,"[1, 776, … 756]",125
2,"[2, 524, … 801]",83
3,"[3, 12, … 948]",75
4,"[4, 439, … 595]",152


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

In [43]:
friends.select(
    pl.col("friends").graph.eigen_centrality().arg_max()
)

friends
u32
327


In [44]:
friends.select(
    # Shortest path to the node at index = 3 (id = 3, the 4th node in this case)
    pl.col("friends").graph.shortest_path_const_cost(target = 3, parallel=True).alias("path")
).head()

path
list[u64]
"[0, 796, … 3]"
"[1, 388, 3]"
"[2, 971, … 3]"
[3]
"[4, 930, … 3]"


In [45]:
relationships = pl.DataFrame({
    "id": range(2000),
    "connections":[[1,2,3,4], [0,2], [4], [0,1,2], [1]] * 400,
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1], [0.5], [0.1, 0.1, 0.1], [0.1]] * 400
}).with_columns(
    pl.col("connections").list.eval(pl.element().cast(pl.UInt64))
)
relationships.head()

id,connections,close-ness
i64,list[u64],list[f64]
0,"[1, 2, … 4]","[0.4, 0.3, … 0.1]"
1,"[0, 2]","[0.1, 1.0]"
2,[4],[0.5]
3,"[0, 1, 2]","[0.1, 0.1, 0.1]"
4,[1],[0.1]


In [46]:
# To go to 1, node at index 0 would rather go to 4 first and then 1.
relationships.select(
    pl.col("connections").graph.shortest_path(
        target = 1,
        cost = pl.col("close-ness"),
        parallel = False
    ).alias("path")
).unnest("path").head()

path,cost
list[u64],f64
"[0, 4, 1]",0.2
[1],0.0
"[2, 4, 1]",0.6
"[3, 1]",0.1
"[4, 1]",0.1


# String Nearest Neighbors

This might be slow for very large vocab / column.

In [47]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"]
})

In [48]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 1,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [49]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [50]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [51]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
